In [1]:
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt

# DATA TREATMENT

In [2]:
columns = [
    'Subject',
    'Measuring station',
    'repetition no.',
    'Neck girth',
    'Back neck point to waist',
    'Upper arm girth R',
    'Back neck point to wrist R',
    'Across back shoulder width',
    'Bust girth',
    'Waist girth',
    'Hip girth',
    'Thigh girth R',
    'Total crotch length',
    'Inside leg height',
]

In [3]:
measures_data = pd.read_csv("data/body_measurements.csv", sep=';')
measures_data = measures_data[columns]
measures_list = ['Neck girth', 'Back neck point to waist', 'Upper arm girth R', 'Back neck point to wrist R', 'Across back shoulder width', 'Bust girth', 'Waist girth', 'Hip girth', 'Thigh girth R', 'Total crotch length', 'Inside leg height']
station_list=measures_data["Measuring station"].unique()

In [4]:
for column in columns:
    if column != 'Measuring station' and column != 'Subject':
        filtered = measures_data[column] > 2000
        measures_data[column].loc[filtered] /= 1000

In [5]:
extra_data = pd.read_csv('data/basic_informations.csv', sep=';')

In [6]:
treated_data = pd.merge(measures_data, extra_data, on='Subject')
treated_data = treated_data.drop(['Pantone', 'BMI (kg/m^2)'], axis=1)

In [7]:
treated_data[treated_data['Sex'] == 'male'].to_pickle('data/male_measures.zip')
treated_data[treated_data['Sex'] == 'female'].to_pickle('data/female_measures.zip')

In [8]:
outliers = []
subjects_list = treated_data['Subject'].unique()
for subject in subjects_list:
    for measure in measures_list:
        temp_data = treated_data[treated_data["Subject"] == subject][["Measuring station", 'Subject','repetition no.', measure]].copy()
        temp_data[measure]
        q1 = temp_data[measure].quantile(0.25)
        q3 = temp_data[measure].quantile(0.75)
        iqr = q3 - q1 #Interquartile range
        fence_low = q1 - (1.5*iqr)
        fence_high = q3 + (1.5*iqr)

        filter = (temp_data[measure] < fence_low) | (temp_data[measure] > fence_high)
        outliers.extend(temp_data[filter].index)
outliers = pd.Series(outliers)

In [9]:
outliers = []
subjects_list = treated_data['Subject'].unique()
for subject in subjects_list:
    for measure in measures_list:
        temp_data = treated_data[treated_data["Subject"] == subject][["Measuring station", 'Subject','repetition no.', measure]].copy()
        zscore = (temp_data[measure] - temp_data[measure].mean()) / (temp_data[measure].std())
        filter = abs(zscore) > 3
        outliers.extend(temp_data[filter].index)
outliers = pd.Series(outliers)
outliers = treated_data.loc[outliers]['Subject'].unique()

In [10]:
#treated_data = treated_data.drop(outliers)
treated_data = treated_data[~treated_data["Subject"].isin(outliers)]

# CALC ICC

In [11]:
their_icc = pd.read_csv('data/their_icc.csv', sep=',', header=0)
their_icc = their_icc.drop(['Measuring station'], axis=1)
their_icc.index = ['Expert measurer 1', 'Expert measurer 2', 'Expert measurer 3', 'Expert measurer 4', 'Expert measurer 5', 'SS20', 'Vitus', 'PortalMX', 'MOVE4D', 'SS@Home', '3DLOOK', 'eM+', '3Davatarbody']
their_icc.columns = measures_list
their_icc.sort_index()


all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in measures_list:
        a = treated_data[treated_data['Measuring station'] == station].reset_index()
        icc = pg.intraclass_corr(data=a, targets='Subject', raters='repetition no.', ratings=measure, nan_policy='omit')['ICC'].loc[0]
        measures_of_station.append(icc)
    all_measures.append(measures_of_station)
our_icc = pd.DataFrame(all_measures, station_list, measures_list).sort_index()

## OUR ICC

In [12]:
our_icc.round(3)

,Neck girth,Back neck point to waist,Upper arm girth R,Back neck point to wrist R,Across back shoulder width,Bust girth,Waist girth,Hip girth,Thigh girth R,Total crotch length,Inside leg height
3DLOOK,0.998,0.990,0.990,0.989,0.987,0.989,0.992,0.992,0.985,0.990,0.996
3Davatarbody,0.984,0.958,0.985,0.963,0.965,0.987,0.996,0.994,0.992,0.972,0.987
Expert measurer 1,0.981,0.908,0.982,0.959,0.938,0.989,0.989,0.988,0.980,0.916,0.945
Expert measurer 2,0.983,0.772,0.980,0.967,0.919,0.991,0.995,0.995,0.983,0.932,0.985
Expert measurer 3,0.979,0.866,0.985,0.954,0.852,0.995,0.997,0.992,0.971,0.970,0.984
Expert measurer 4,0.985,0.913,0.979,0.961,0.955,0.990,0.991,0.992,0.984,0.913,0.977
Expert measurer 5,0.980,0.847,0.994,0.983,0.945,0.990,0.995,0.994,0.988,0.951,0.983
MOVE4D,0.996,0.965,0.996,0.986,0.975,0.994,0.997,0.999,0.998,0.974,0.993
PortalMX,0.988,0.985,0.988,0.972,0.894,0.995,0.999,0.998,0.992,0.976,0.989
SS20,0.992,0.967,0.991,0.989,0.981,0.998,0.995,0.995,0.986,0.968,0.980


## ICC DIFFERENCE

In [13]:
abs(our_icc - their_icc).round(3)

,Neck girth,Back neck point to waist,Upper arm girth R,Back neck point to wrist R,Across back shoulder width,Bust girth,Waist girth,Hip girth,Thigh girth R,Total crotch length,Inside leg height
3DLOOK,0.000,0.002,0.004,0.002,0.002,0.001,0.007,0.001,0.001,0.003,0.001
3Davatarbody,0.001,0.010,0.001,0.009,0.003,0.001,0.000,0.001,0.001,0.005,0.003
Expert measurer 1,0.001,0.003,0.000,0.005,0.000,0.001,0.001,0.000,0.000,0.009,0.003
Expert measurer 2,0.000,0.003,0.000,0.002,0.000,0.001,0.000,0.000,0.001,0.003,0.001
Expert measurer 3,0.001,0.001,0.002,0.002,0.001,0.000,0.000,0.000,0.003,0.007,0.001
Expert measurer 4,0.001,0.001,0.000,0.003,0.004,0.000,0.000,0.000,0.001,0.002,0.002
Expert measurer 5,0.000,0.037,0.000,0.001,0.009,0.000,0.000,0.000,0.002,0.006,0.005
MOVE4D,0.000,0.007,0.000,0.002,0.002,0.000,0.001,0.000,0.000,0.004,0.002
PortalMX,0.000,0.001,0.001,0.002,0.001,0.000,0.000,0.001,0.001,0.001,0.002
SS20,0.000,0.001,0.001,0.005,0.000,0.000,0.000,0.001,0.004,0.003,0.000


# CALC SEM

In [14]:
their_sem = pd.read_csv('data/their_sem.csv', sep=';', header=None)
their_sem.index = ['Expert measurer 1', 'Expert measurer 2', 'Expert measurer 3', 'Expert measurer 4', 'Expert measurer 5', 'SS20', 'Vitus', 'PortalMX', 'MOVE4D', 'SS@Home', '3DLOOK', 'eM+', '3Davatarbody']
their_sem = their_sem.drop(0, axis=1)
their_sem.columns = measures_list
their_sem.sort_index()


all_measures = []
for station in station_list:
    measures_of_station = []
    for measure in measures_list:
        a = treated_data[treated_data['Measuring station'] == station].reset_index()
        icc = pg.intraclass_corr(data=a, targets='Subject', raters='repetition no.', ratings=measure, nan_policy='omit')['ICC'].loc[1]
        sem = a[measure].std() * np.sqrt(1-icc)
        measures_of_station.append(sem)
    all_measures.append(measures_of_station)
our_sem = pd.DataFrame(all_measures, station_list, measures_list).sort_index()

## OUR SEM

In [15]:
our_sem.round(3)

,Neck girth,Back neck point to waist,Upper arm girth R,Back neck point to wrist R,Across back shoulder width,Bust girth,Waist girth,Hip girth,Thigh girth R,Total crotch length,Inside leg height
3DLOOK,1.121,2.389,3.100,3.561,3.044,10.923,10.962,7.849,6.370,5.452,2.902
3Davatarbody,4.703,5.320,4.388,8.201,7.063,11.621,8.724,7.722,5.340,8.933,4.935
Expert measurer 1,5.351,9.883,5.502,9.710,8.138,11.800,13.497,11.428,8.945,21.920,10.747
Expert measurer 2,5.460,12.584,6.068,8.270,9.498,12.886,10.090,7.620,9.431,19.722,5.953
Expert measurer 3,5.908,10.168,5.394,9.323,12.135,9.691,7.620,9.832,12.063,12.678,6.014
Expert measurer 4,5.357,9.574,6.350,9.253,7.533,13.122,13.363,9.445,8.835,19.283,6.888
Expert measurer 5,4.537,11.803,3.217,5.787,7.508,10.535,8.917,7.451,6.295,14.387,6.128
MOVE4D,2.303,5.025,2.504,4.843,5.874,7.945,6.807,2.754,2.683,8.460,3.234
PortalMX,3.903,3.675,4.602,7.310,8.150,7.252,4.215,3.658,5.479,7.626,4.434
SS20,4.614,7.465,4.549,5.648,5.457,6.901,10.500,6.941,8.156,13.863,6.595


## SEM DIFFERENCE

In [16]:
abs(our_sem - their_sem).round(3)

,Neck girth,Back neck point to waist,Upper arm girth R,Back neck point to wrist R,Across back shoulder width,Bust girth,Waist girth,Hip girth,Thigh girth R,Total crotch length,Inside leg height
3DLOOK,0.079,0.011,0.800,0.239,0.456,0.323,5.438,0.249,0.630,1.548,0.098
3Davatarbody,0.103,0.020,0.088,0.201,0.163,0.321,0.124,0.122,0.040,0.333,0.135
Expert measurer 1,0.049,0.117,0.002,0.110,0.138,0.100,0.003,0.172,0.255,0.620,0.613
Expert measurer 2,0.160,0.084,0.068,0.070,0.098,0.186,0.110,0.020,0.031,0.122,0.147
Expert measurer 3,0.108,0.168,0.106,0.077,0.135,0.191,0.380,0.032,0.537,0.822,0.014
Expert measurer 4,0.057,0.174,0.050,0.047,0.167,0.032,0.263,0.255,0.035,0.083,0.212
Expert measurer 5,0.063,1.303,0.083,0.313,0.392,0.235,0.783,0.349,0.195,1.813,0.228
MOVE4D,0.003,0.025,0.004,0.157,0.226,0.045,0.307,0.054,0.083,0.060,0.134
PortalMX,0.003,0.075,0.098,0.190,0.550,0.048,0.115,0.042,0.021,0.774,0.166
SS20,0.114,0.265,0.649,0.148,0.057,0.101,0.100,1.059,0.944,0.537,0.095


# CALC PSD

In [17]:
def calculate_psd(measure, phase):
    data = treated_data[treated_data['Phase'] == phase]
    f1 = data['repetition no.'] == 1
    f2 = data['repetition no.'] == 2
    data = data[ f1 | f2 ][['Subject', 'Measuring station', 'repetition no.', measure]]
    data = data.pivot_table(values=measure, index=['Subject', 'repetition no.'], columns="Measuring station")
    data = data.dropna()

    column_wise = []
    for column1 in data.columns:
        row_wise = []
        for column2 in data.columns:
            if column1 == column2:
                row_wise.append(np.nan)
            else:
                d1 = data[column1]
                d2 = data[column2]
                var = (d1-d2).var()
                sem1 = our_sem[measure].loc[column1]**2
                sem2 = our_sem[measure].loc[column2]**2
                value = np.sqrt(var + (sem1/2) + (sem2/2))
                row_wise.append(value)
        column_wise.append(row_wise)
    data = pd.DataFrame(column_wise, index=data.columns, columns=data.columns)
    data.index.name = None
    return data

In [18]:
calculate_psd(measure='Neck girth', phase=1).round(1)

,Expert measurer 1,Expert measurer 2,Expert measurer 3,Expert measurer 4,SS20,Vitus
Expert measurer 1,NaN,9.5,10.7,10.8,24.0,22.7
Expert measurer 2,9.5,NaN,9.5,9.9,23.6,22.2
Expert measurer 3,10.7,9.5,NaN,11.0,24.2,22.9
Expert measurer 4,10.8,9.9,11.0,NaN,23.0,22.0
SS20,24.0,23.6,24.2,23.0,NaN,13.1
Vitus,22.7,22.2,22.9,22.0,13.1,NaN


In [19]:
calculate_psd(measure='Neck girth', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,PortalMX,SS@Home,eM+
3DLOOK,NaN,25.5,22.1,19.7,25.9,23.7,29.7,30.1
3Davatarbody,25.5,NaN,16.5,16.7,15.0,13.0,19.4,17.4
Expert measurer 1,22.1,16.5,NaN,10.2,10.5,11.7,16.0,16.7
Expert measurer 5,19.7,16.7,10.2,NaN,12.4,12.8,17.6,17.9
MOVE4D,25.9,15.0,10.5,12.4,NaN,9.2,13.3,14.8
PortalMX,23.7,13.0,11.7,12.8,9.2,NaN,15.3,14.9
SS@Home,29.7,19.4,16.0,17.6,13.3,15.3,NaN,19.7
eM+,30.1,17.4,16.7,17.9,14.8,14.9,19.7,NaN


In [20]:
calculate_psd(measure='Upper arm girth R', phase=1).round(1)

,Expert measurer 1,Expert measurer 2,Expert measurer 3,Expert measurer 4,SS20,Vitus
Expert measurer 1,NaN,12.5,10.7,13.9,34.1,17.7
Expert measurer 2,12.5,NaN,11.5,13.0,33.9,16.5
Expert measurer 3,10.7,11.5,NaN,12.4,32.4,16.4
Expert measurer 4,13.9,13.0,12.4,NaN,35.3,18.4
SS20,34.1,33.9,32.4,35.3,NaN,30.7
Vitus,17.7,16.5,16.4,18.4,30.7,NaN


In [21]:
calculate_psd(measure='Upper arm girth R', phase=2).round(1)

,3DLOOK,3Davatarbody,Expert measurer 1,Expert measurer 5,MOVE4D,PortalMX,SS@Home,eM+
3DLOOK,NaN,15.5,19.2,18.2,16.9,19.1,16.9,27.5
3Davatarbody,15.5,NaN,16.8,15.5,14.1,16.7,15.7,28.0
Expert measurer 1,19.2,16.8,NaN,11.9,9.4,11.6,14.0,29.7
Expert measurer 5,18.2,15.5,11.9,NaN,8.2,9.4,15.6,31.4
MOVE4D,16.9,14.1,9.4,8.2,NaN,8.1,12.5,29.7
PortalMX,19.1,16.7,11.6,9.4,8.1,NaN,15.0,33.1
SS@Home,16.9,15.7,14.0,15.6,12.5,15.0,NaN,27.3
eM+,27.5,28.0,29.7,31.4,29.7,33.1,27.3,NaN
